# Introduction

Adeano-associated viruses (AAVs) are used in gene therapy as genome vehicle. They are found in different tissues on different animals as well as humans. These are call wild type ones. 

In order to increase the efficiency of gene therapy, we need good AAV capsdid, ie, their covers. The goodness are measured according to the following criteria

1. tissue tropism: whether they can attach to a specific organs such as lunch, heart, or liver
2. thermal stability: whether the AAV capsid are stable under heat
3. immunological reaction: whether the AAV capsid invoke the reaction of the host immune system.

Directed evolution approach in protein engineering are costly and slow, we want to accelerate it by using ML-methods to recommend potential candidates for wet-lab experiments.

Our vision is to build an end-to-end platform to design protein sequence → start with AAV capsid

At the moment we have experimented with: 

- We are building a testbed to experiment with different protein engineering methods in silico. They include three main part
    - Oracles: predictive models that can be trained on real dataset
    - Ground Truth: These are landscapes of (x,y), where x and y are sequence and fitness score, respectively. These are collected using wet-lab experiments.  They are used to benchmark different recommenders
    - Recommenders: These algorithms use oracles to navigate the ground truth landscape. Note that we do not have access to the landscape in a real experiment.
- AAV tropism dataset from "Comprehensive AAV capsid fitness landscape reveals a viral gene and enables machine-guided design" by Ogden, Pierce J and Kelsic, Eric D and Sinai, Sam and Church, George M.
- We try different methods
    - CbAS, DbAS.: the main idea cast the exploration problem as an optimization problem, where you wan to maximize the expectation of the likelihood a given sequence has desired properties. The expectation is computed with respect to the generative distribution.


The production of AAVs capsid are controled by the **cap** section on the virus genome. 


### DbAS

In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import editdistance
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pprint
import numpy as np
import json

In [6]:
import sys
sys.path.insert(0, '../')

import flexs
from flexs import baselines
import flexs.utils.sequence_utils as s_utils

### Additive AAV tropism capsid

Let's build a landscape (ground truth) for liver

In [32]:
additive_aav_problem = flexs.landscapes.additive_aav_packaging.registry()['liver']
additive_aav_landscape = flexs.landscapes.AdditiveAAVPackaging(**additive_aav_problem['params'])
additive_aav_wt = additive_aav_landscape.wild_type
additive_aav_landscape = flexs.landscapes.AdditiveAAVPackaging(**additive_aav_problem['params'])

pprint.pprint(additive_aav_problem)
print(f'WT: {additive_aav_wt}, fitness: {additive_aav_landscape.get_fitness([additive_aav_wt])}')

{'params': {'end': 540, 'phenotype': 'liver', 'start': 450}}
WT: PSGTTTQSRLQFSQAGASDIRDQSRNWLPGPCYRQQRVSKTSADNNNSEYSWTGATKYHLNGRDSLVNPGPAMASHKDDEEKFFPQSGVL, fitness: [0.5]


Let's explore the landscape. We need three ingredients

1. Oracle
2. Generative model
3. Explorer

we start from the wild type

In [44]:
starting_sequence = additive_aav_wt

alphabet = s_utils.AAS

cnn = baselines.models.CNN(len(starting_sequence), alphabet=alphabet,
                         num_filters=32, hidden_size=100, loss='MSE')

vae = baselines.explorers.VAE(len(starting_sequence), alphabet=alphabet, epochs=10, verbose=False)

cbas_explorer = baselines.explorers.CbAS(
    model=cnn,
    generator=vae,
    cycle_batch_size=100,
    rounds=10,
    starting_sequence=starting_sequence,
    sequences_batch_size=100,
    model_queries_per_batch=2000,
    alphabet=alphabet,
)

In [48]:
cbas_sequences, metadata = cbas_explorer.run(landscape=additive_aav_landscape)

round: 0, top: 0.5, time: 0.000312s


/Users/totucuong/miniconda3/envs/cinference/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


round: 1, top: 0.5101216714222102, time: 0.378895s


/Users/totucuong/miniconda3/envs/cinference/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


0 100
100 100
200 100
300 100
400 100
500 100
600 100
700 100
800 100
900 100
1000 100
1100 100
1200 100
1300 100
1400 100
1500 100
1600 100
1700 100
1800 100
1900 100
round: 2, top: 0.5101216714222102, time: 233.911361s


...

0 100
100 100
200 100
300 100
400 100
500 100
600 100
700 100
800 100
900 100
1000 100
1100 100
1200 100
1300 100
1400 100
1500 100
1600 100
1700 100
1800 100
1900 100
round: 8, top: 0.5101216714222102, time: 263.599728s


/Users/totucuong/miniconda3/envs/cinference/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


0 100
100 100
200 100
300 100
400 100
500 100
600 100
700 100
800 100
900 100
1000 100
1100 100
1200 100
1300 100
1400 100
1500 100
1600 100
1700 100
1800 100
1900 100
round: 9, top: 0.5101216714222102, time: 242.450165s


/Users/totucuong/miniconda3/envs/cinference/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


0 100
100 100
200 100
300 100
400 100
500 100
600 100
700 100
800 100
900 100
1000 100
1100 100
1200 100
1300 100
1400 100
1500 100
1600 100
1700 100
1800 100
1900 100
round: 10, top: 0.5101216714222102, time: 254.542627s


In [50]:
metadata

{'run_id': '16:45:52-11/11/2020',
 'exp_name': 'cbas_Q=0.7_generator=VAE_latent_dim=2_intermediate_dim=250',
 'model_name': 'CNN_hidden_size_100_num_filters_32',
 'landscape_name': 'AdditiveAAVPackaging_phenotype=liver',
 'rounds': 10,
 'sequences_batch_size': 100,
 'model_queries_per_batch': 2000}